In [15]:
import csv
from collections import deque
import elasticsearch
from elasticsearch import helpers
import json
import pandas as pd

In [16]:
def readMovies():
    csvfile = open('ml-latest-small/movies.csv', 'r')

    reader = csv.DictReader( csvfile )

    titleLookup = {}

    for movie in reader:
            titleLookup[movie['movieId']] = movie['title']

    return titleLookup

In [17]:
def readRatings():
    csvfile = open('ml-latest-small/ratings.csv', 'r')

    titleLookup = readMovies()

    reader = csv.DictReader( csvfile )
    for line in reader:
        rating = {}
        rating['user_id'] = int(line['userId'])
        rating['movie_id'] = int(line['movieId'])
        rating['title'] = titleLookup[line['movieId']]
        rating['rating'] = float(line['rating'])
        rating['timestamp'] = int(line['timestamp'])
        yield rating

In [18]:
es = elasticsearch.Elasticsearch()

In [20]:
#delete if index already defined
es.indices.delete(index="ratings",ignore=404)
settings={
        "mappings": {
                "rating": {
                        "properties": {
                                "title": {

                                        "type":"text",
                                        "fielddata":True,
                                        "fields": {
                                                "raw": {
                                                        "type":"keyword"
                                                }
                                        }
                                }
                        }
                }
        }
}
es.indices.create(index="ratings", ignore=400, body=settings)
#import data to index
deque(helpers.parallel_bulk(es,readRatings(),index="ratings",doc_type="rating"), maxlen=0)
es.indices.refresh()

{'_shards': {'failed': 0, 'successful': 149, 'total': 217}}

Verification

In [21]:
def search(es_object, index_name, search):
    res = es_object.search(index=index_name, body=search)
    data=(res['hits']['hits'])
    for i in data:
        print(i)

In [22]:
if es is not None:
    search_object = {'query': {'match': {'title': 'star'}}}
    search(es, 'ratings', json.dumps(search_object))

{'_index': 'ratings', '_type': 'rating', '_id': '0O0ac2YBSMzvaMU4m0-O', '_score': 4.6106668, '_source': {'user_id': 91, 'movie_id': 68358, 'title': 'Star Trek (2009)', 'rating': 4.5, 'timestamp': 1448798186}}
{'_index': 'ratings', '_type': 'rating', '_id': 'Pu0ac2YBSMzvaMU4q3Iy', '_score': 4.6106668, '_source': {'user_id': 159, 'movie_id': 800, 'title': 'Lone Star (1996)', 'rating': 4.5, 'timestamp': 1183519437}}
{'_index': 'ratings', '_type': 'rating', '_id': 'Du0ac2YBSMzvaMU4p2kT', '_score': 4.6106668, '_source': {'user_id': 134, 'movie_id': 68358, 'title': 'Star Trek (2009)', 'rating': 4.0, 'timestamp': 1361244848}}
{'_index': 'ratings', '_type': 'rating', '_id': 'dO0ac2YBSMzvaMU4mkow', '_score': 4.6106668, '_source': {'user_id': 78, 'movie_id': 68358, 'title': 'Star Trek (2009)', 'rating': 4.0, 'timestamp': 1344469772}}
{'_index': 'ratings', '_type': 'rating', '_id': 'UO0ac2YBSMzvaMU4kjdI', '_score': 4.6106668, '_source': {'user_id': 48, 'movie_id': 68358, 'title': 'Star Trek (2009